In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta

In [2]:
path = "C:\\Users\\Martyna\\Studia\\Bazy danych\\Projekt\\games_to_choose.csv"
games = pd.read_csv(path) #all games

In [7]:
def choose_games(games_to_choose, num_of_games):
    """
    Returns list with ID of games which the shop will be able to sale or rent.
    
    Arguments
    ---------
    games_to_choose_from_df - data frame with all games
    num_of_games - num of games' titles that the shop will be able to sale or rent
    """
    tournament_games = games_to_choose[games_to_choose["Tournament_game"] == 1]
    not_tournament_games = games_to_choose[games_to_choose["Tournament_game"] == 0]
    games_sample = not_tournament_games.sample(num_of_games - 5)
    not_tournament_id = list(games_sample["Game_ID"])
    tournament_id = list(tournament_games["Game_ID"])
    all_games = not_tournament_id + tournament_id
    return all_games

In [8]:
all_games = choose_games(games, 50) #game

In [13]:
def choose_products(games_to_choose_from, num_of_products):
    """
    Returns data frame with products that are available on the first day.
    
    Arguments
    ---------
    games_to_choose_from_df - data frame with all games that the shop will be able to sale or rent
    num_of_products - num of productst that the shop will stock on the first day
    """
    
    not_tournament_id = games_to_choose_from[-5:]
    tournament_products = [not_tournament_id[i] for i in np.arange(5)]*5
    for_rent_tournament = np.zeros(25)
    for_sale_tournament = np.zeros(25)
    for_tournament_tournament = [1 for i in np.arange(25)]
    
    not_tournament_products = random.choices(games_to_choose_from, k=num_of_products - 25)
    for_tournament_nt = [0 for i in np.arange(num_of_products - 25)]
    supply_date= ['2022-04-29' for i in np.arange(num_of_products)]
    
    for_sale_nt = np.zeros(num_of_products - 25)
    for_rent_nt = np.zeros(num_of_products - 25)

    for i in np.arange(num_of_products - 25):
        rand = random.randint(0, 1)
        for_sale_nt[i] = rand
        if for_sale_nt[i] == 0:
            for_rent_nt[i] = 1
        else:
            for_rent_nt[i] = 0
            
    for_rent = list(for_rent_tournament) + list(for_rent_nt)
    for_sale = list(for_sale_tournament) + list(for_sale_nt)
    for_tournament = for_tournament_tournament + for_tournament_nt
    all_games_products = tournament_products + not_tournament_products
    
    product_df = pd.DataFrame()
    product_df["Product_ID"] = np.arange(num_of_products)
    product_df["For_rent"] = for_rent
    product_df["For_sale"] = for_sale
    product_df["For_tournament"] = for_tournament
    product_df["Supply_date"] = supply_date
    product_df["Game_ID"] = all_games_products
    product_df["Purchase_ID"] = np.nan

    product_df["Supply_date"] = pd.to_datetime(product_df["Supply_date"])
    product_df["For_rent"] = pd.to_numeric(product_df["For_rent"], downcast='integer')
    product_df["For_sale"] = pd.to_numeric(product_df["For_sale"], downcast='integer')
    
    return product_df

In [14]:
products_df = choose_products(all_games, 500)

In [19]:
def product_df_time(product_df, first_date): 
    
    rental_id = [0]
    rental_date = []
    return_date_expected = []
    return_date_actual = []
    
    relation_id = [0]
    product_id_rel = []
    rental_id_rel = []
    
    product_and_return = {}
    
    purchase_id = [0]
    purchase_date = []
    
    num_to_rent = np.random.poisson(10, 365)
    num_to_sale = np.random.poisson(10, 365)
    
    num_to_supply = np.random.poisson(10, 365)
    
    j = 0
    for days_count in np.arange(1, 366):
        
        new_date = first_date + timedelta(days = int(days_count))
        
        games_rentable = product_df[product_df["For_rent"] == 1]
        
        num_of_games_to_rent = np.random.poisson(1/1.3, num_to_rent[j]) #ile gier na wypożyczenie?
        
        while sum(num_of_games_to_rent) > len(games_rentable): 
            num_of_games_to_rent = num_of_games_to_rent[:-1]
            
        random_return = np.random.random(len(num_of_games_to_rent))
        
        games_to_rent = games_rentable["Product_ID"].sample(sum(num_of_games_to_rent)) #jakie gry wypożyczono?
        
        k = 0
        i = 0
        for rent in num_of_games_to_rent:
            
            games_to_this_rent = np.array(games_to_rent)[k:k+rent]
            
            available_products = []
            
            for game in games_to_this_rent: #sprawdzamy, czy produkty są dostępne
                if game not in list(product_and_return.keys()) or product_and_return[game] < new_date:
                    available_products.append(game)
             
            num_of_products = len(available_products)
            
            
            if num_of_products > 0:
                
                rental_id.append(rental_id[-1]+1)
                rental_date.append(new_date)
                return_date_expected.append(new_date + timedelta(days = 3))
                
                if random_return[i] < 0.5:
                    return_date_actual.append(return_date_expected[-1])
                    product_and_return[game] = return_date_expected[-1]

                else:
                    return_date = new_date + timedelta(days = np.random.geometric(1/3))
                    return_date_actual.append(return_date)
                    product_and_return[game] = return_date
                    
                relation_id = relation_id + [relation_id[-1] + i for i in np.arange(1, num_of_products+1)]
                rental_id_rel = rental_id_rel + [rental_id[-1]]*num_of_products
                product_id_rel = product_id_rel + available_products
            
            k += rent
            i += 1
        
        #zakup gier
        
        games_salable = product_df[(product_df["For_sale"] == 1) & (pd.isna(product_df["Purchase_ID"]))]
        num_of_games_to_sale = np.random.geometric(1/1.3, num_to_sale[j]) #ile gier na wypożyczenie?
        
        while sum(num_of_games_to_sale) > len(games_salable): 
            num_of_games_to_sale = num_of_games_to_sale[:-1]
                    
        games_to_sale = games_salable["Product_ID"].sample(sum(num_of_games_to_sale)) #jakie gry wypożyczono?
         
        k = 0    
        
        for purchase in num_of_games_to_sale:
            
            games_to_this_purchase = np.array(games_to_sale)[k:k+purchase] 
            
            purchase_id.append(purchase_id[-1]+1)
            purchase_date.append(new_date)
          #  print(len(product_df))
            for game_product in games_to_this_purchase:
               # print(product_df.loc[game_product, "Purchase_ID"]) # tu się psuje
                product_df.loc[game_product, "Purchase_ID"] = purchase_id[-1]
               # print(product_df.loc[game_product])
                
            k += purchase
            
        games_to_supply = random.choices(all_games, k = num_to_supply[j])
        
        max_product_id = max(np.array(product_df["Product_ID"]))
        new_df = pd.DataFrame()
        new_df["Product_ID"] = np.arange(max_product_id + 1, max_product_id + num_to_supply[j] + 1)
        new_df["For_sale"] = [1 for _ in np.arange(num_to_supply[j])]
        new_df["For_rent"] = [0 for _ in np.arange(num_to_supply[j])]
        new_df["For_tournament"] = [0 for _ in np.arange(num_to_supply[j])]
        new_df["Supply_date"] = [new_date for _ in np.arange(num_to_supply[j])]
        new_df["Game_ID"] = games_to_supply
        new_df["Purchase_ID"] = np.nan
        
        new_df["Supply_date"] = pd.to_datetime(new_df["Supply_date"])
        new_df["For_rent"] = pd.to_numeric(new_df["For_rent"], downcast='integer')
        new_df["For_sale"] = pd.to_numeric(new_df["For_sale"], downcast='integer')


        product_df = pd.concat([product_df, new_df], ignore_index=True)  
        
        j += 1
                    
            
        
        
    
    rental_df = pd.DataFrame({"Rental_ID":rental_id[1:], "Rental_date":rental_date, "Return_date_expected":return_date_expected, "Return_date_actual":return_date_actual})
    rental_product_rel = pd.DataFrame({"Relation_ID":relation_id[1:], "Product_ID":product_id_rel, "Rental_ID":rental_id_rel})
    purchase_df = pd.DataFrame({"Purchase_ID":purchase_id[1:], "Purchase_date":purchase_date})
    
    
    return rental_df, rental_product_rel, purchase_df, product_df

In [21]:
rental_df, rental_product_rel, purchase_df, product_df = product_df_time(products_df, products_df["Supply_date"].loc[0])